# Creating Custom Players

In [ ]:
from maverick import (
    Game,
    Player,
    ActionType,
    PlayerAction,
)
from maverick.utils import estimate_holding_strength, find_highest_scoring_hand


class CustomPlayerExample(Player):
    
    def decide_action(
        self,
        *,
        game: Game,
        valid_actions: list["ActionType"],
        min_raise_amount: int,
        call_amount: int,
        **_
    ) -> "PlayerAction":
        # ---------- Information used to make decision ----------
        
        amount_to_call = max(0, int(game.state.current_bet) - int(self.state.current_bet))  # Amount needed to call if the action is CALL
        min_raise_by = int(min_raise_amount)                            # Minimum amount to raise BY if the action is RAISE
        
        street_name = game.state.street.name                     # This is the current street (e.g., PRE_FLOP, FLOP, TURN, RIVER)
        button_position = game.state.button_position             # Position of the dealer button
        pot = game.state.pot                                     # Total chips in the pot
        current_bet = game.state.current_bet                     # Current highest bet in this round
        small_blind = game.state.small_blind                     # Small blind amount
        big_blind = game.state.big_blind                         # Big blind amount
        community_cards = game.state.community_cards             # List of community cards on the table
        stack = self.state.stack                                 # Player's current stack
        current_bet_player = self.state.current_bet              # Player's current bet in this round
        private_cards = self.state.holding.cards                 # Player's private cards

        # ----------- Hand strength evaluation ----------
        
        strongest_hand, strongest_hand_type, strongest_hand_score = find_highest_scoring_hand(
            private_cards, community_cards, n_private=None
        )
        
        # strongest_hand: list[Card] = best possible hand cards
        # strongest_hand_type: HandType = type of the best hand (e.g., STRAIGHT_FLUSH)
        # strongest_hand_score: int = numerical score of the best hand

        hand_prob = estimate_holding_strength(
            private_cards,
            community_cards=community_cards,
            n_private=None,
            n_simulations=1000,
            n_players=len(game.state.get_players_in_hand()),
        )
        
        # hand_prob: float = estimated probability of having the best hand (equity)

        # ---------- Decision logic (example, this is what is personal to a player) ----------
        
        if ActionType.RAISE in valid_actions and hand_prob > 0.7:
            raise_amount = min(self.state.stack, amount_to_call + min_raise_by * 2)
            action = PlayerAction(ActionType.RAISE, raise_amount)
            print(f"     Decision: RAISE to {raise_amount}")
        elif ActionType.CALL in valid_actions and hand_prob > 0.4:
            action = PlayerAction(ActionType.CALL, amount_to_call)
            print(f"     Decision: CALL {amount_to_call}")
        elif ActionType.CHECK in valid_actions:
            action = PlayerAction(ActionType.CHECK)
            print("     Decision: CHECK")
        else:
            action = PlayerAction(ActionType.FOLD)
            print("     Decision: FOLD")
        
        # Return the chosen action
        return action